In [ ]:
# !pip install -r ../requirements.txt

In [2]:
# !pip uninstall virny -y

In [ ]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [23]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [25]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [26]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSEmploymentDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_preprocessing_intervention

## Define Input Variables

In [27]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
FAIR_INTERVENTION_PARAMS_LST = [0.0 + 0.1 * i for i in range(11)]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'folk_ny_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [28]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [29]:
import uuid

custom_table_fields_dct = {
    'session_uuid': str(uuid.uuid4()),
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  0b71b2a8-030f-44df-a027-5304af77e2ba


## Initialize custom objects

In [30]:
data_loader = ACSEmploymentDataset(state=['NY'], year=2018, with_nulls=False,
                                   subsample_size=20_000, subsample_seed=42)
data_loader.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,5,4,0,1,2,1,2,2,2,1,1,2,1,1,15,42
1,1,4,0,1,2,1,2,2,2,2,1,0,1,2,21,59
2,2,4,0,1,2,2,2,2,2,2,1,0,4,4,19,78
3,5,4,0,1,2,1,2,2,2,1,1,10,1,4,14,19
4,1,2,0,1,2,1,2,2,2,1,1,0,1,1,17,87


In [31]:
data_loader.X_data.shape

(20000, 16)

## Run experiment iterations

### Experiment iteration 1

In [32]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [33]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             with_tuning=True,
                                             # with_tuning=False,
                                             # tuned_params_df_path=tuned_params_df_path,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

2023-07-05 17:49:25 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.0, 0.2, 0.4, 0.6]',
 'model_init_seed': 100,
 'session_uuid': '0b71b2a8-030f-44df-a027-5304af77e2ba'}




2023-07-05 17:49:25 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')


Multiple alphas:   0%|          | 0/4 [00:00<?, ?it/s]

...

[CV 3/3; 1/2] START n_estimators=100............................................
[CV 3/3; 1/2] END n_estimators=100; Accuracy_Score: (test=0.734) F1_Score: (test=0.725) total time=   3.0s
[CV 1/3; 1/2] START n_estimators=100............................................
[CV 1/3; 1/2] END n_estimators=100; Accuracy_Score: (test=0.744) F1_Score: (test=0.735) total time=   3.3s
[CV 2/3; 1/2] START n_estimators=100............................................
[CV 2/3; 1/2] END n_estimators=100; Accuracy_Score: (test=0.750) F1_Score: (test=0.742) total time=   3.6s
[CV 3/3; 2/2] START n_estimators=200............................................
[CV 3/3; 2/2] END n_estimators=200; Accuracy_Score: (test=0.736) F1_Score: (test=0.727) total time=   4.7s
[CV 2/3; 2/2] START n_estimators=200............................................
[CV 2/3; 2/2] END n_estimators=200; Accuracy_Score: (test=0.756) F1_Score: (test=0.749) total time=   5.8s
[CV 1/3; 2/2] START n_estimators=200........................

Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

[CV 3/3; 1/2] START n_estimators=100............................................
[CV 3/3; 1/2] END n_estimators=100; Accuracy_Score: (test=0.732) F1_Score: (test=0.723) total time=   2.7s
[CV 2/3; 1/2] START n_estimators=100............................................
[CV 2/3; 1/2] END n_estimators=100; Accuracy_Score: (test=0.750) F1_Score: (test=0.743) total time=   3.4s
[CV 1/3; 1/2] START n_estimators=100............................................
[CV 1/3; 1/2] END n_estimators=100; Accuracy_Score: (test=0.749) F1_Score: (test=0.740) total time=   4.3s
[CV 3/3; 2/2] START n_estimators=200............................................
[CV 3/3; 2/2] END n_estimators=200; Accuracy_Score: (test=0.733) F1_Score: (test=0.724) total time=   5.6s
[CV 2/3; 2/2] START n_estimators=200............................................
[CV 2/3; 2/2] END n_estimators=200; Accuracy_Score: (test=0.755) F1_Score: (test=0.748) total time=   5.9s
[CV 1/3; 2/2] START n_estimators=200........................

### Experiment iteration 2

In [34]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__144936.csv',
                          'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__145444.csv',
                          'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__150103.csv',
                          'tuning_results_Folktables_NY_2018_Employment_alpha_0.6_20230705__150645.csv']
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [35]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

2023-07-05 18:13:51 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0, 0.2, 0.4, 0.6]',
 'intervention_param': 0.6,
 'model_init_seed': 200,
 'session_uuid': '0b71b2a8-030f-44df-a027-5304af77e2ba'}




2023-07-05 18:13:52 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')


Multiple alphas:   0%|          | 0/4 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

Path for tuned params:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance/results/mult_repair_levels/tuning_results_Folktables_NY_2018_Employment_alpha_0.6_20230705__150645.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': 201, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]